In [80]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [87]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_1Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [82]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [83]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [84]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [85]:
def save_report(file_path, df):
    print(df)
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)

In [86]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [88]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(1):
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

Prompt==> {"payload":"Problem: Write a function in python to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n."} 
 {"id":"cmpl-2caa8b5af5504d3da0d6b0bb8e87cd0c","object":"text_completion","created":435549,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nn = int(input(\"Enter a non-negative integer: \"))\nprint(\"The factorial of\", n, \"is\", factorial(n))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":118,"completion_tokens":73}}
Execution time: 3.668709 seconds 


Prompt==> {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-10d0598b16dd4fa2a370715037059993","object":"text_completion"

Prompt==> {"payload":"Problem: Write a function in java to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."} 
 {"id":"cmpl-09deea52c5264a06add6bd18011ccbda","object":"text_completion","created":435597,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}
Execution time: 0.094861 seconds 


Prompt==> {"payload":"Problem: Write a function in java to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome."} 
 {"id":"cmpl-959f63c6df1c42728dba5644af0f0b40","object":"text_completion","created":435597,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":

Prompt==> {"payload":"Problem: Write a function in c++ to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num."} 
 {"id":"cmpl-ab6af2cfbdd94c00a7800bb36bc2aef1","object":"text_completion","created":435632,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n#include <iostream>\nusing namespace std;\n\nint sumOfDigits(int num) {\n    int sum = 0;\n    while (num != 0) {\n        sum += num % 10;\n        num /= 10;\n    }\n    return sum;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a positive integer: \";\n    cin >> num;\n    cout << \"Sum of digits: \" << sumOfDigits(num) << endl;\n    return 0;\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":49,"total_tokens":181,"completion_tokens":132}}
Execution time: 7.427408 seconds 


Prompt==> {"payload":"Problem: Write a function in c++ to

Prompt==> {"payload":"Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number."} 
 {"id":"cmpl-a1d26b3aade1483bb13e488d0dd8995b","object":"text_completion","created":435685,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nfunction fibonacci(n) {\n    if (n <= 1) {\n        return n;\n    } else {\n        return fibonacci(n - 1) + fibonacci(n - 2);\n    }\n}\n\nconsole.log(fibonacci(10));","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":124,"completion_tokens":76}}
Execution time: 4.481008 seconds 


Prompt==> {"payload":"Problem: Write a function in javascript to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams."} 
 {"id":"cmpl-a88952af2777484ea54603c0bd24f6a3","object":"text_completion","created"

Prompt==> {"payload":"Problem: Write a function in Ruby to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-df8dbc8af9264e4a90ea2b233ea1f7f9","object":"text_completion","created":435720,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef binary_search(arr, target)\n  low = 0\n  high = arr.length - 1\n\n  while low <= high\n    mid = (low + high) / 2\n\n    if arr[mid] == target\n      return mid\n    elsif arr[mid] < target\n      low = mid + 1\n    else\n      high = mid - 1\n    end\n  end\n\n  return -1\nend\n\narr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\ntarget = 5\nputs binary_search(arr, target)  # Output: 4\n\ntarget = 11\nputs binary_search(arr, target)  # Output: -1","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":53,"total_tokens":235,"completion_tokens

Prompt==> {"payload":"Problem: Implement form validation in CSS to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid."} 
 {"id":"cmpl-de318ccdd0ff4399a7760c4d949602c3","object":"text_completion","created":435737,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}
Execution time: 0.095761 seconds 


Prompt==> {"payload":"Problem: Create an image slider in CSS that displays images in a sequence with next and previous buttons.Input: Array of image URLs.Output: None (update the displayed image)."} 
 {"id":"cmpl-9b4bd74a340b4838a390feaf507267ed","object":"text_completion","created":435737,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"ind

Prompt==> {"payload":"Problem: Create a function in react to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element)."} 
 {"id":"cmpl-6b170d9688a9466093b48ae09aa114e0","object":"text_completion","created":435739,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":50,"total_tokens":51,"completion_tokens":1}}
Execution time: 0.098704 seconds 


Prompt==> {"payload":"Problem: Implement an autocomplete feature in angular for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions)."} 
 {"id":"cmpl-4714643428c0477fbceebfa53c7dfb1b","object":"text_completion","created":435739,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c

Prompt==> {"payload":"Problem: Implement form validation in jQuery to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid."} 
 {"id":"cmpl-931998b44cb94811845208f06610f18d","object":"text_completion","created":435740,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}
Execution time: 0.095707 seconds 


Prompt==> {"payload":"Problem: Create an image slider in jQuery that displays images in a sequence with next and previous buttons.Input: Array of image URLs.Output: None (update the displayed image)."} 
 {"id":"cmpl-6f9465266c514b5db8965668c0735b37","object":"text_completion","created":435740,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))